In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

try:
    sys.path.append(str(Path().cwd().parent))
except IndexError:
    pass

# from util... import *

In [ ]:
STATS_FILE = Path.cwd().parent / "util" / "quality-bd-bus.csv"

df = pd.read_csv(STATS_FILE)

In [ ]:
def plot_bd_rate(df):
    """Plot single BD rate"""
    plt.plot(df.bitrate, df.psnr)
    plt.scatter(df.bitrate, df.psnr)
    # plt.plot(df.bitrate, df.ssim)
    # plt.plot(df.bitrate, df.vmaf)
    labels = [int(qp) for qp in df.qp]
    labels.reverse()
    plt.title("BD-rate of bus (bitrate [kbit/s]) + filter k = 3, sigma = 1.5")
    # plt.title("BD-rate of bus (bitrate [kbit/s]) + qp 38 prep")
    plt.ylabel("PSNR [db]")
    plt.xlabel(labels)
    plt.savefig("bus-bd-filter.png")
    # plt.savefig("bus-bd-bus-38.png")


def plot_bd_rate_gauss(df):
    """Plot single BD for multiple filters"""
    for key, grp in df.groupby(["k_size"]):
        plt.plot(grp["bitrate"], grp["psnr"], label=f"kernel size: {int(key)}")
        plt.scatter(grp["bitrate"], grp["psnr"])
    labels = [float(sigma) for sigma in pd.unique(df.sigma)]
    labels.reverse()
    plt.xlabel(labels)
    plt.legend()
    plt.title("BD-rate of bus (bitrate [kbit/s]) Gauss filter + QP 24")
    plt.savefig("bus-bd-gauss.png")

In [ ]:
plot_bd_rate(df)